In [ ]:
# %pip install python-dotenv openai
# %pip install --upgrade openai

In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
from datetime import datetime
from IPython.display import HTML
import IPython
import re

In [ ]:
load_dotenv()  # take environment variables from .env.
               # you should set OPENAI_API_KEY="sk-xxxxx" in .env in current working_dir.

In [ ]:
client = OpenAI()

role_definition = """
You're my English tutor.
You have a list of colors: "Scarlet,Crimson,Maroon,Burgundy,Coral,Tangerine,Apricot,Burnt Orange,Mustard,Gold,Amber,Lemon,Emerald,Olive,Lime,Jade,Navy,Azure,Cobalt,Turquoise,Lavender,Mauve,Plum,Violet,Fuchsia,Salmon,Rose,Blush,Sienna,Bronze,Taupe,Beige,Charcoal,Slate,Silver,Pewter,Ivory,Pearl,Alabaster,Jet,Onyx,Ebony".
Whatever I say, you'll randomly choose one of colors in the list, 
1. tell me the color's name, and give the pronunciation of the word in ipa.
2. give me the hex code of the color;
3. give me the name of the color in Chinese, 
4. tell me what basic color's variation it is.
5. explain to me it's origin, where I would see the color most often, in both English and Chinese; and please surround English explanation with tripple ```

"""

user_prompt = """
hi
"""

rspd_translation = client.chat.completions.create(
  model="gpt-4",
  messages=[
    {
        "role": "system", 
         "content": role_definition
    },
    {
        "role": "user", 
        "content": user_prompt
    }
  ]
)

colors = re.findall(r'#[0-9a-fA-F]{3}(?:[0-9a-fA-F]{3})?', rspd_translation.choices[0].message.content)
html_str = f"<div style='width:100px; height:100px; background-color:{colors[0]};'></div>"
color_explanation = re.findall(r'```(.*?)```', rspd_translation.choices[0].message.content)
print("")
display(HTML(html_str))
print("")
print(rspd_translation.choices[0].message.content)

In [ ]:
speech_file_path = f"{datetime.now().strftime("%Y%m%d_%H%M%S")}_color.mp3"
voice_performer = "alloy"
# alloy, echo, fable, onyx, nova, and shimmer, the last two of which are femail voices.

rspd_audio = client.audio.speech.create(
  model="tts-1",
  voice=voice_performer,
  input=color_explanation[0]
)

rspd_audio.stream_to_file(speech_file_path)

IPython.display.Audio(speech_file_path)